In [ ]:

# path to nn model & tokenizer
model_path = '../models/tc_model_1_1680891414_tt_1680888903.h5'
tokenizer_path = '../models/tokenizer_1680888903.pickle'

In [ ]:
def load_deta_tokenizer(model_path, tokenizer_path):
    with open(tokenizer_path, 'rb') as file:
        tokenizer = pickle.load(file)
        
    model = keras.models.load_model(model_path)
    
    return model, tokenizer

In [ ]:
The method compresses the data to the list of the following tupples: (id, name, address, lattitude, longtitude, web page, google page, description, interest vector)

In [ ]:
def vectorize_places(places, model, tokenizer):
    places_with_details = []
    maxlen = model.layers[0].input_length
    
    for place in places:
        place_id = place[0]
        place_name = place[1]
        place_types = place[2]
        place_lat = place[3]
        place_lng = place[4]
        place_summary = None
        place_reviews = []
        place_website = None
        place_address = None
        place_url = None

        response = execute_details_request(place_id)
        response_data = response.json()['result']

        if 'editorial_summary' in response_data:
            place_summary = response_data['editorial_summary']['overview']
        if 'reviews' in response_data:
            for review in response_data['reviews']:
                place_reviews.append(review['text'])
        if 'website' in response_data:
            place_website = response_data['website']
        if 'formatted_address' in response_data:
            place_address = response_data['formatted_address']
        if 'url' in response_data:
            place_url = response_data['url']
    
        if place_summary is None and len(place_reviews) == 0:
            continue
            
        description = ''
        for place_type in place_types:
            description += (place_type.replace('_', ' ')) + ' '
            
        if place_summary is not None:
            description += place_summary + ' '
            
        for review in place_reviews:
            description += review.replace('\n', '').replace('\\', '') + ' '
        
        description = description.replace('  ', ' ')
        sequences = tokenizer.texts_to_sequences([description])
        x = pad_sequences(sequences, maxlen=maxlen)
        place_vector = model.predict(x, verbose=0)[0]
        
        places_with_details.append((
            place_id, place_name, place_types, 
            place_lat, place_lng, place_address, 
            place_website, place_url, place_summary,
            place_reviews, place_vector
        ))
    
    return places_with_details